In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

Outline of  build a deep NN  
<img src="images/final outline.png" style="width:800px;height:500px;">

**Exercise**: Implement initialization for an L-layer Neural Network. 

In [2]:
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network,
                including input layer (Layer 0) and output layer(Layer L-1),L is the length of the list
                like [2,4,1] means two input units and one hidden layer with 4 units and one output unit
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    np.random.seed(3)
    parameter={}
    for i in range(1,len(layer_dims)):
        parameter['W'+str(i)] = np.random.randn(layer_dims[i],layer_dims[i-1]) * 0.01
        parameter['b'+str(i)] = np.zeros(shape=(layer_dims[i],1))
    
        assert( parameter['W'+str(i)].shape == (layer_dims[i],layer_dims[i-1]))
        assert( parameter['b'+str(i)].shape == (layer_dims[i],1))
    
    return parameter

In [3]:
initialize_parameters_deep([5,4,3])

{'W1': array([[ 0.01788628,  0.0043651 ,  0.00096497, -0.01863493, -0.00277388],
        [-0.00354759, -0.00082741, -0.00627001, -0.00043818, -0.00477218],
        [-0.01313865,  0.00884622,  0.00881318,  0.01709573,  0.00050034],
        [-0.00404677, -0.0054536 , -0.01546477,  0.00982367, -0.01101068]]),
 'W2': array([[-0.01185047, -0.0020565 ,  0.01486148,  0.00236716],
        [-0.01023785, -0.00712993,  0.00625245, -0.00160513],
        [-0.00768836, -0.00230031,  0.00745056,  0.01976111]]),
 'b1': array([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.]]),
 'b2': array([[ 0.],
        [ 0.],
        [ 0.]])}

### Linear Forward
Exercise: Build the linear part of forward propagation.
### Linear Activation Forward
Exercise: Implement the forward propagation of the LINEAR->ACTIVATION layer. Mathematical relation is:  A[l]=g(Z[l])=g(W[l]A[l−1]+b[l])A[l]=g(Z[l])=g(W[l]A[l−1]+b[l])  where the activation "g" can be sigmoid() or relu(). Use linear_forward() and the correct activation function.

In [10]:
def linear_forward(A,W,b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    linear_cache -- a python dictionary containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    Z = np.dot(W,A)+ b
    
    linear_cache= (A,W,b)
    
    assert(Z.shape == (W.shape[0],A.shape[1]))
    return Z,linear_cache

def linear_activation_forward(A_prev,W,b,activation):
    '''
    returns: 
    A:output of the activation function
    cache:
    '''
    if activation == 'sigmoid':
        Z,linear_cache = linear_forward(A_prev,W,b)
        A,activation_cache = sigmoid(Z)
    elif activation == 'relu':
        Z,linear_cache = linear_forward(A_prev,W,b)
        A,activation_cache = relu(Z)
    
#     cache={}
#     cache['A_prev']= A_prev
#     cache['Z'] = Z
#     cache['A'] = A
#     cache['W'] = W
#     cache['b'] = b

    cache = (linear_cache, activation_cache)
    
    assert(A.shape==(W.shape[0],A_prev.shape[1]))
    return A,cache

Exercise: Implement the forward propagation of L-layer NN

In [11]:
def forward_prop(X,parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep() ,contains W and b
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing: each cache has two sub-caches(tuples):
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """
    L= len(parameters) // 2 +1 # number of layers (including input layer)
    A = X
    caches=[]
    # for layer 1  to layer l-2
    for k in range(1,L-1):
        A_prev = A
        A,cache = linear_activation_forward(A_prev,parameters['W'+str(k)],parameters['b'+str(k)],activation= 'relu')
        caches.append(cache)
        
    AL ,cache = linear_activation_forward(A,parameters['W'+str(L-1)],parameters['b'+str(L-1)],activation='sigmoid')
    caches.append(cache)
    #AL is the probability of the samples being positive 
    
    assert(AL.shape==(1,X.shape[1]))
    return AL,caches

In [12]:
X, parameters = L_model_forward_test_case()
AL, caches = forward_prop(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[ 0.17007265  0.2524272 ]]
Length of caches list = 2


In [13]:
print(caches)

[((array([[ 1.62434536, -0.61175641],
       [-0.52817175, -1.07296862],
       [ 0.86540763, -2.3015387 ],
       [ 1.74481176, -0.7612069 ]]), array([[ 0.3190391 , -0.24937038,  1.46210794, -2.06014071],
       [-0.3224172 , -0.38405435,  1.13376944, -1.09989127],
       [-0.17242821, -0.87785842,  0.04221375,  0.58281521]]), array([[-1.10061918],
       [ 1.14472371],
       [ 0.90159072]])), array([[-2.77991749, -2.82513147],
       [-0.11407702, -0.01812665],
       [ 2.13860272,  1.40818979]])), ((array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 2.13860272,  1.40818979]]), array([[ 0.50249434,  0.90085595, -0.68372786]]), array([[-0.12289023]])), array([[-1.58511248, -1.08570881]]))]


### Compute cost function   ##and calculate da^[L]

In [7]:
def compute_cost(AL,Y):
    m = len(AL)  # num of examples  or Y.shape[1]
    cost = (-1/m)*mnp.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    
    assert(np.squeeze(cost).shape==())
    return cost

### Back_propagation


For layer $l$, the linear part is: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (followed by an activation).

Suppose you have already calculated the derivative $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. You want to get $(dW^{[l]}, db^{[l]} dA^{[l-1]})$.

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center> **Figure 4** </center></caption>

The three outputs $(dW^{[l]}, db^{[l]}, dA^{[l]})$ are computed using the input $dZ^{[l]}$.Here are the formulas you need:
$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$

In [16]:
def linear_backward(dZ,linear_cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    linear_cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev,W,b = linear_cache
   
    m = A_prev.shape[1] # num of samples
    dW = 1/m * np.dot(dZ, A_prev.T)
    db = 1/m * np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [17]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[ 0.50629448]]


In [20]:
def linear_activation_backward(dA,cache,activation):
    '''
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    '''
    linear_cache , activation_cache = cache
    
    if activation == 'sigmoid':
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
    elif activation == 'relu':
        dZ = relu_backward(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
    
    return dA_prev,dW,db

In [21]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


**Exercise:** combine the parts together to a back_propagation model (L layer)  

### 6.3 - L-Model Backward 

Now you will implement the backward function for the whole network. Recall that when you implemented the `forward_prop` function, at each iteration, you stored a cache which contains (X,W,b, and z). In the back propagation module, you will use those variables to compute the gradients. Therefore, in the `backward_prop` function, you will iterate through all the hidden layers backward, starting from layer $L$. On each step, you will use the cached values for layer $l$ to backpropagate through layer $l$. Figure 5 below shows the backward pass. 


<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center>  **Figure 5** : Backward pass  </center></caption>

** Initializing backpropagation**:
To backpropagate through this network, we know that the output is, 
$A^{[L]} = \sigma(Z^{[L]})$. Your code thus needs to compute `dAL` $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$.
To do so, use this formula (derived using calculus which you don't need in-depth knowledge of):
```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL
```

You can then use this post-activation gradient `dAL` to keep going backward. As seen in Figure 5, you can now feed in `dAL` into the LINEAR->SIGMOID backward function you implemented (which will use the cached values stored by the L_model_forward function). After that, you will have to use a `for` loop to iterate through all the other layers using the LINEAR->RELU backward function. You should store each dA, dW, and db in the grads dictionary. To do so, use this formula : 

$$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

For example, for $l=3$ this would store $dW^{[l]}$ in `grads["dW3"]`.


In [24]:
def backward_prop(AL,Y,cache):
    '''
    AL : predict value of the output
    Y : label 
    *****caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    '''
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    L= len(caches)
    grads={}
#     m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    current_cache = caches[-1] #output_layer of cache
    # output layer
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL,current_cache, activation = "sigmoid")
    for k in reversed(range(L-1)):
        current_cache = caches[k]
        dA_prev_temp, dW_temp, db_temp=linear_activation_backward(dAL,current_cache,activation='relu')
        grads["dA" + str(k+1)] = dA_prev_temp
        grads["dW" + str(k+1)] = dW_temp
        grads["db" + str(k+1)] = db_temp
        
    return grads

### Finally, update the parameters

In [25]:
def update_parameters(parameters,grads,lr):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    lr-- learning rate
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    L = len(parameters) // 2
    for k in range(L):
        parameters['W'+str(k+1)] = parameters['W'+str(k+1)] - lr* grads['dW'+str(k+1)]
        parameters['b'+str(k+1)] = parameters['b'+str(k+1)] - lr* grads['db'+str(k+1)]
    
    return parameters